In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)BASE_PATH = '/content/drive/MyDrive/Sound'
TRAIN_PATH = BASE_PATH  

# Sampling rate
sr = 22050
# Load duration- 20s out of 30s
load_duration = 2
# Samples for corresponding load duration
TOTAL_SAMPLES = load_duration * sr
# Number of slices to be created of the load duration
NUM_SLICES = 1

SAMPLES_PER_SLICE = int(TOTAL_SAMPLES / NUM_SLICES)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

--2022-05-15 09:55:24--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.89.236
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.89.236|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  46.4MB/s    in 5.6s    

2022-05-15 09:55:30 (42.7 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]



In [ ]:
!unzip annotations_trainval2017.zip -d annotations_trainval2017

Archive:  annotations_trainval2017.zip
  inflating: annotations_trainval2017/annotations/instances_train2017.json  
  inflating: annotations_trainval2017/annotations/instances_val2017.json  
  inflating: annotations_trainval2017/annotations/captions_train2017.json  
  inflating: annotations_trainval2017/annotations/captions_val2017.json  
  inflating: annotations_trainval2017/annotations/person_keypoints_train2017.json  
  inflating: annotations_trainval2017/annotations/person_keypoints_val2017.json  


In [ ]:
!pip install pycocotools
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import time
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [ ]:
dataDir='annotations_trainval2017'
dataType='train2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)

In [ ]:
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=26.11s)
creating index...
index created!


In [ ]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

COCO supercategories: 
electronic accessory sports animal food vehicle furniture kitchen appliance person outdoor indoor


In [ ]:
# For cats
our_categories = ['dog']
imgIds = []
for cat_name in our_categories:
  # get all images containing given categories, select one at random
  catIds = coco.getCatIds(catNms=cat_name)
  imgIds += coco.getImgIds(catIds=catIds)

imgIds = list(set(imgIds))
imgs = coco.loadImgs(imgIds)

In [ ]:
# initialize COCO api for caption annotations
annFile = '{}/annotations/captions_{}.json'.format(dataDir,dataType)
coco_caps=COCO(annFile)

loading annotations into memory...
Done (t=1.14s)
creating index...
index created!


In [ ]:
!pip install gtts

In [ ]:
!mkdir DLProjSpeechAnnotationsDogs

In [ ]:
from gtts import gTTS #Import Google Text to Speech
from IPython.display import Audio, clear_output #Import Audio method from IPython's Display Class

def full_tts(idx, count):
  for img in imgs[idx:min(idx+50, len(imgs))]:
    annIds = coco_caps.getAnnIds(imgIds = img['id'])
    captions = coco_caps.loadAnns(annIds)[0:5]
    clear_output(wait = True)
    print(count+1, " / ", len(imgs))
    time.sleep(10)
    for i in range(len(captions)):
      caption = captions[i]['caption']
      tts = gTTS(caption, slow = True)
      save_file_name = './DLProjSpeechAnnotationsDogs/' + img['file_name'][:-4] + '_' + str(i) + '.wav'
      tts.save(save_file_name)
    count += 1
  idx += 50
  return idx, count

In [ ]:
count = 0
idx = 0
for idx in range(0, len(imgs), 50):
  idx, count = full_tts(idx, count)

2093  /  4385
